Данный файл подготовлен в качестве ipynb-адаптации [текста о модуле `ipaddress`](https://proglib.io/p/koncepciya-ip-adresov-na-primere-python-modulya-ipaddress-2020-08-07), подготовленного автором Библиотеки программиста `eFusion`. Текст является переводом публикации Брэда Соломона [Learn IP Address Concepts With Python's ipaddress Module](https://realpython.com/python-ipaddress-module/).

# Создание и преобразование объектов IPv4Address

В самом грубом представлении IP-адрес – это просто число. Поэтому классическое представление в виде 4 октетов можно преобразовывать к другим видам.

In [1]:
from ipaddress import IPv4Address
addr = IPv4Address("220.14.9.37")
addr

IPv4Address('220.14.9.37')

Из целого числа:

In [2]:
IPv4Address(3691907365)

IPv4Address('220.14.9.37')

Из байтовой строки:

In [3]:
IPv4Address(b"\xdc\x0e\t%")

IPv4Address('220.14.9.37')

Обратное преобразование:

In [4]:
int(addr)

3691907365

In [5]:
addr.packed

b'\xdc\x0e\t%'

Объекты `IPv4Address` хэшируемы:

In [6]:
hash(IPv4Address("220.14.9.37"))

7026503466999608753

In [7]:
num_connections = {IPv4Address("220.14.9.37"): 2,
                   IPv4Address("100.201.0.4"): 16,
                   IPv4Address("8.240.12.2"): 4}

Класс IPv4Address также реализует методы, позволяющие сравнивать адреса:

In [8]:
IPv4Address("220.14.9.37") > IPv4Address("8.240.12.2")

True

Раз можно сравнивать, значит, можно и сортировать:

In [9]:
for a in sorted(num_connections.keys()):
    print(a)

8.240.12.2
100.201.0.4
220.14.9.37


# Нотация CIDR

In [10]:
from ipaddress import IPv4Network
net = IPv4Network("192.4.2.0/24")
net.num_addresses

256

Длина префикса:

In [11]:
net.prefixlen

24

Битовая маска сети для побитовых операций:

In [12]:
net.netmask

IPv4Address('255.255.255.0')

Проверка вхождения в сеть:

In [13]:
IPv4Address("192.4.2.12") in net

True

In [14]:
IPv4Address("192.4.20.2") in net

False

Широковещательный адрес:

In [15]:
net.network_address

IPv4Address('192.4.2.0')

Чтобы определить число адресов в подсети:

In [16]:
net = IPv4Network("100.64.0.0/10")
net.num_addresses

4194304

Маска для побитовых операций:

In [17]:
net.netmask

IPv4Address('255.192.0.0')

# Перебор IP-адресов в цикле

In [18]:
net = IPv4Network("192.4.2.0/28")

for addr in net:
    print(addr)

192.4.2.0
192.4.2.1
192.4.2.2
192.4.2.3
192.4.2.4
192.4.2.5
192.4.2.6
192.4.2.7
192.4.2.8
192.4.2.9
192.4.2.10
192.4.2.11
192.4.2.12
192.4.2.13
192.4.2.14
192.4.2.15


Так можно создать генератор, выдающий адреса, исключая сетевые и широковещательные:

In [19]:
h = net.hosts()
type(h)

generator

In [20]:
next(h)

IPv4Address('192.4.2.1')

In [21]:
next(h)

IPv4Address('192.4.2.2')

# Подсети

In [22]:
small_net = IPv4Network("192.0.2.0/28")

In [23]:
big_net = IPv4Network("192.0.0.0/16")

Проверяем, что одна сеть является подмножеством другой:

In [24]:
small_net.subnet_of(big_net)

True

In [25]:
big_net.supernet_of(small_net)

True

Находим подсети. При необходимости с заменой префикса:

In [26]:
for sn in net.subnets():
    print(sn)

192.4.2.0/29
192.4.2.8/29


In [27]:
for sn in net.subnets(new_prefix=28):
    print(sn)

192.4.2.0/28


# Методы проверки специальных диапазонов IP-адресов

In [28]:
IPv4Address("10.243.156.214") in IPv4Network("10.0.0.0/8")

True

In [29]:
timesync_addr = IPv4Address("169.254.169.123")
timesync_addr.is_link_local

True

In [30]:
IPv4Address("10.243.156.214").is_private

True

In [31]:
IPv4Address("127.0.0.1").is_loopback

True

In [32]:
[i for i in dir(IPv4Address) if i.startswith("is_")]
['is_global',
 'is_link_local',
 'is_loopback',
 'is_multicast',
 'is_private',
 'is_reserved',
 'is_unspecified']

['is_global',
 'is_link_local',
 'is_loopback',
 'is_multicast',
 'is_private',
 'is_reserved',
 'is_unspecified']

# Создание собственных классов на основе ipaddress

In [33]:
addr = IPv4Address("220.14.9.37")
addr._ip   # атрибут, соответствующий целочисленному представлению IP

3691907365

In [34]:
from ipaddress import IPv4Address

class MyIPv4(IPv4Address):
    def __and__(self, other: IPv4Address):
        if not isinstance(other, (int, IPv4Address)):
            raise NotImplementedError
        return self.__class__(int(self) & int(other))

In [35]:
addr = MyIPv4("100.127.40.32")
mask = MyIPv4("255.192.0.0")

In [36]:
addr & mask

MyIPv4('100.64.0.0')

In [37]:
addr & 0xffc00000

MyIPv4('100.64.0.0')

In [38]:
import re
from ipaddress import IPv4Address

class MyIPv4(IPv4Address):
    @property
    def binary_repr(self, sep=".") -> str:
        """Представляет IPv4 в виде 4 блоков по 8 бит."""
        return sep.join(f"{i:08b}" for i in self.packed)  # 8 строка

    @classmethod
    def from_binary_repr(cls, binary_repr: str):
        """Создает IPv4 из двоичного представления."""
        i = int(re.sub(r"[^01]", "", binary_repr), 2)  # 14 строка
        return cls(i)

Так реализована двусторонняя конвертация:

In [39]:
MyIPv4("220.14.9.37").binary_repr

'11011100.00001110.00001001.00100101'

In [40]:
MyIPv4("255.255.0.0").binary_repr  # Маска для префикса /16 

'11111111.11111111.00000000.00000000'

In [41]:
MyIPv4.from_binary_repr("11011100 00001110 00001001 00100101")

MyIPv4('220.14.9.37')